In [498]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [499]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [500]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [501]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle

scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = Credentials.from_service_account_file("credentials.json", scopes= scopes)
client = gspread.authorize(creds)

sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"TablaFinImpacto"])-3

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1501271018.py:26: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn=int(RN_df.loc[:,"TablaFinImpacto"])-3


## Misión

In [502]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle

scopes = [
    "https://www.googleapis.com/auth/spreadsheets"
]

creds = Credentials.from_service_account_file("credentials.json", scopes= scopes)
client = gspread.authorize(creds)

sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

TablaFinImpacto=libro.get_worksheet(7)
TablaFinImpacto_d = TablaFinImpacto.get_all_values()
TablaFinImpacto_df=pd.DataFrame(TablaFinImpacto_d)
TablaFinImpacto_df=TablaFinImpacto_df.iloc[2:,[0,4]]
TablaFinImpacto_df.columns=["ID", "Respuesta"]
df_final = TablaFinImpacto_df.iloc[rn:,:]

acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}
def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
df_final['Respuesta'] = df_final['Respuesta'].astype(str)
nltk.download('all')
text = list(df_final['Respuesta'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)

df_final['Respuesta'] = corpus

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1449633345.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1449633345.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Respuesta'] = df_final['Respuesta'].astype(str)
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\pablo\AppData\Roaming\nlt

In [503]:
with open ("NBbuenoTD1.pkl","rb") as modelo_1:
    modelo_td1 = pickle.load(modelo_1)

prediccion = modelo_td1.predict(df_final['Respuesta'])
prediccion_f = prediccion + 2
df_final["Clasificación"]=prediccion_f

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[0,1]]
Rubrica_df.columns=["Mision", "Clasificación"]
Rubrica_df["Mision"]=Rubrica_df["Mision"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[0,-2,-1]]
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
last_row = len(Puntaje.get_all_values()) + 1  # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"A{last_row}", values)
nuevo_valor=int(RN_df.loc[:,"TablaFinImpacto"]) + len(pred_s)
RN.update("G2", [[nuevo_valor]])

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\264407527.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Clasificación"]=prediccion_f
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\264407527.py:33: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Puntaje.update(f"A{last_row}", values)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\264407527.py:34: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  nuevo_valor=int(RN_df.loc[:,"TablaFinImpacto"]) + len(pred_s)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\264407

{'spreadsheetId': '1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY',
 'updatedRange': 'Row_number!G2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

## Coherencia del diseño programático

In [504]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
print(rn)

233


C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\171902243.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2


In [505]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","DiseñoProyecto","DuraciónProyectoMeses","ContribucionIntervencionFin"]]
df.columns=["ID","Diseno","Duración","Contribucion"]
df_final=df.iloc[rn:,:]


acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Coherencia', how='inner')
# df_final= pd.merge(df_sfinal,df, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,2]]
df_final['Diseno'] = df_final['Diseno'].apply(quitar_acentos)
df_final['Diseno'] = df_final['Diseno'].astype(str)
df_final['Contribucion'] = df_final['Contribucion'].apply(quitar_acentos)
df_final['Contribucion'] = df_final['Contribucion'].astype(str)
df_final["Duración"] = df_final["Duración"].str.extract("(\d+)")  # Extrae solo los números
df_final["Duración"] = pd.to_numeric(df_final["Duración"])
# df_final = df_final.dropna()
text = list(df_final['Diseno'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Diseno'] = corpus
text = list(df_final['Contribucion'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Contribucion'] = corpus
df_final

<>:36: SyntaxWarning: invalid escape sequence '\d'
<>:36: SyntaxWarning: invalid escape sequence '\d'
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\471732412.py:36: SyntaxWarning: invalid escape sequence '\d'
  df_final["Duración"] = df_final["Duración"].str.extract("(\d+)")  # Extrae solo los números
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\471732412.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Diseno'] = df_final['Diseno'].apply(quitar_acentos)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\471732412.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

,ID,Diseno,Duración,Contribucion
234,7076548,primero realizamos diagnostico participativo e...,6,intervenciones proyecto atencion medica integr...


In [506]:
with open("tfidf1_model_td2.pkl", "rb") as f:  
    tfidf1_td2_loaded = pickle.load(f)  

with open("tfidf2_model_td2.pkl", "rb") as f:  
    tfidf2_td2_loaded = pickle.load(f)  

with open("scaler_model_td2.pkl", "rb") as f:  
    scaler_td2_loaded = pickle.load(f)  

X_text1 = df_final["Diseno"]
X_text2 = df_final["Contribucion"]
X_num = df_final[["Duración"]]

X_text1_t = tfidf1_td2_loaded.transform(X_text1)
X_text2_t = tfidf2_td2_loaded.transform(X_text2)
X_num_t = scaler_td2_loaded.transform(X_num)

X_combined = hstack([X_text1_t, X_text2_t, X_num_t])

with open ("RFbuenoTD2.pkl","rb") as modelo_2:
    modelo_td2 = pickle.load(modelo_2)

prediccion = modelo_td2.predict(X_combined)
df_final["Clasificación"]= prediccion
df_final
Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[2,3]]
Rubrica_df.columns=["Coherencia", "Clasificación"]
Rubrica_df["Coherencia"]=Rubrica_df["Coherencia"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,3]]
Puntaje_df.columns=["ID","Coherencia"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"D{last_row-len(pred_s)}", values)
# nuevo_valor=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"]) + len(pred_s)
# RN.update("A2", [[nuevo_valor]])

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1081211086.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Clasificación"]= prediccion
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1081211086.py:57: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Puntaje.update(f"D{last_row-len(pred_s)}", values)


{'spreadsheetId': '1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y',
 'updatedRange': 'Sheet1!D277:E277',
 'updatedRows': 1,
 'updatedColumns': 2,
 'updatedCells': 2}

## Problema derecho

In [507]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn1=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn2=int(RN_df.loc[:,"TablaCausas"])-3
rn3=int(RN_df.loc[:,"TablaEfectos"])-3

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1068380983.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn1=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1068380983.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn2=int(RN_df.loc[:,"TablaCausas"])-3
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1068380983.py:12: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn3=int(RN_df.loc[:,"TablaEfectos"])-3


In [508]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
TablaCausas=libro.get_worksheet(9)
TablaEfectos=libro.get_worksheet(8)
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","PrincipalDerechoHumano","ProblemaPrincipal","DatosCausas","QuéPasaríaConLasPersonasQueAtiendeSiSuOrganizaciónNoInterviniera"]]
df.columns=["ID","DerechoHumano","Problema","DatosCausas","NoIntervencion"]
df=df.iloc[rn:,:]
TablaCausas=libro.get_worksheet(9)
TablaCausas_d = TablaCausas.get_all_values()
TablaCausas_df=pd.DataFrame(TablaCausas_d)
TablaCausas_df.columns=TablaCausas_df.iloc[0,:]
TablaCausas_df=TablaCausas_df.iloc[2:,:]
TablaCausas_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Causas"]]
TablaCausas_df["Causas"] = TablaCausas_df["Causas"].str.replace('\n', ' ', regex=True)
df2=TablaCausas_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Causas"]]
df2.columns=["ID","Causas"]
df2=df2.iloc[rn2:,:]
TablaEfectos=libro.get_worksheet(8)
TablaEfectos_d = TablaEfectos.get_all_values()
TablaEfectos_df=pd.DataFrame(TablaEfectos_d)
TablaEfectos_df.columns=TablaEfectos_df.iloc[0,:]
TablaEfectos_df=TablaEfectos_df.iloc[2:,:]
TablaEfectos_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Efectos"]]
TablaEfectos_df["Efectos"] = TablaEfectos_df["Efectos"].str.replace('\n', ' ', regex=True)
df3=TablaEfectos_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Efectos"]]
df3.columns=["ID","Efectos"]
df3=df3.iloc[rn3:,:]
df_2= pd.merge(df,df2, on="ID", how="inner")
df_final= pd.merge(df_2,df3, on="ID", how="inner")


acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['DerechoHumano'] = df_final['DerechoHumano'].apply(quitar_acentos)
df_final['DerechoHumano'] = df_final['DerechoHumano'].astype(str)
df_final['Problema'] = df_final['Problema'].apply(quitar_acentos)
df_final['Problema'] = df_final['Problema'].astype(str)
df_final['DatosCausas'] = df_final['DatosCausas'].apply(quitar_acentos)
df_final['DatosCausas'] = df_final['DatosCausas'].astype(str)
df_final['NoIntervencion'] = df_final['NoIntervencion'].apply(quitar_acentos)
df_final['NoIntervencion'] = df_final['NoIntervencion'].astype(str)
df_final['Causas'] = df_final['Causas'].apply(quitar_acentos)
df_final['Causas'] = df_final['Causas'].astype(str)
df_final['Efectos'] = df_final['Efectos'].apply(quitar_acentos)
df_final['Efectos'] = df_final['Efectos'].astype(str)

text = list(df_final['DerechoHumano'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DerechoHumano'] = corpus
text = list(df_final['Problema'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Problema'] = corpus
text = list(df_final['DatosCausas'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DatosCausas'] = corpus
text = list(df_final['NoIntervencion'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['NoIntervencion'] = corpus
text = list(df_final['Causas'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Causas'] = corpus
text = list(df_final['Efectos'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Efectos'] = corpus

In [509]:
with open("tfidf1_model_td3.pkl", "rb") as f:  
    tfidf1_td3_loaded = pickle.load(f)  

with open("tfidf2_model_td3.pkl", "rb") as f:  
    tfidf2_td3_loaded = pickle.load(f)  

with open("tfidf3_model_td3.pkl", "rb") as f:  
    tfidf3_td3_loaded = pickle.load(f)  

with open("tfidf4_model_td3.pkl", "rb") as f:  
    tfidf4_td3_loaded = pickle.load(f)  

with open("tfidf5_model_td3.pkl", "rb") as f:  
    tfidf5_td3_loaded = pickle.load(f)  

with open("tfidf6_model_td3.pkl", "rb") as f:  
    tfidf6_td3_loaded = pickle.load(f)  


X_text1=df_final['DerechoHumano'] = df_final['DerechoHumano'].apply(quitar_acentos)
X_text2=df_final['Problema'] = df_final['Problema'].apply(quitar_acentos)
X_text3=df_final['DatosCausas'] = df_final['DatosCausas'].apply(quitar_acentos)
X_text4=df_final['NoIntervencion'] = df_final['NoIntervencion'].apply(quitar_acentos)
X_text5=df_final['Causas'] = df_final['Causas'].apply(quitar_acentos)
X_text6=df_final['Efectos'] = df_final['Efectos'].apply(quitar_acentos)

X_text1_t=tfidf1_td3_loaded.transform(X_text1)
X_text2_t=tfidf2_td3_loaded.transform(X_text2)
X_text3_t=tfidf3_td3_loaded.transform(X_text3)
X_text4_t=tfidf4_td3_loaded.transform(X_text4)
X_text5_t=tfidf5_td3_loaded.transform(X_text5)
X_text6_t=tfidf6_td3_loaded.transform(X_text6)

X_combined = hstack([X_text1_t, X_text2_t, X_text3_t, X_text4_t, X_text5_t, X_text6_t])

with open ("RFbuenoTD3.pkl","rb") as modelo_3:
    modelo_td3 = pickle.load(modelo_3)

prediccion = modelo_td3.predict(X_combined)
df_final["Clasificación"]= prediccion

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[4,5]]
Rubrica_df.columns=["ProblemaDerecho", "Clasificación"]
Rubrica_df["ProblemaDerecho"]=Rubrica_df["ProblemaDerecho"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.17,
    2: 2.33,
    3: 3.50,
    4: 4.67,
    5: 5.83,
    6: 7
}

Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,6]]
Puntaje_df.columns=["ID","ProblemaDerecho"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
pred_s
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"G{last_row-len(pred_s)}", values)

# # df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='ProblemaDerecho', how='inner')
# # df_final= pd.merge(df_sfinal,df_3, on="ID", how="inner")
# # df_final = df_final.iloc[:,[0,3,4,5,6,7,8,2]]


C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\3176429033.py:73: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Puntaje.update(f"G{last_row-len(pred_s)}", values)


{'spreadsheetId': '1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y',
 'updatedRange': 'Sheet1!G277:H277',
 'updatedRows': 1,
 'updatedColumns': 2,
 'updatedCells': 2}

## Perspectiva de atención

In [510]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn

C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\2479684632.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2


233

In [511]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","SusActividadesImplicanElInvolucramientoDeLosTitularesDeResponsabilidadesfamiliasComunidadesOrganizacionesDeLaSociedadCivilOSCSindicatosYDeObligacionesinstitucionesPúblicasDelEstadoYLosGobernantesQuéSeRealizaConEllosasQuéBuscaGenerarEnLosMismos"]]
SolicitudDeInversiónSocialFocal_df.columns=["ID", "Respuesta"]
df_final=SolicitudDeInversiónSocialFocal_df.iloc[rn:,:]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Perspectiva', how='inner')
# df_final= pd.merge(df_sfinal,SolicitudDeInversiónSocialFocal_df, on="ID", how="inner")
# df_final=df_final.iloc[:,[-1,-2]]
df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
df_final['Respuesta'] = df_final['Respuesta'].astype(str)
df_final['Respuesta'] = df_final['Respuesta'].str.replace(r'[^\w\s]', '', regex=True)
nltk.download('all')
text = list(df_final['Respuesta'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):
    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Respuesta'] = corpus

X = df_final['Respuesta']


C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\3647332631.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\3647332631.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Respuesta'] = df_final['Respuesta'].astype(str)
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\3647332631.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [512]:
with open ("RF_TD4.pkl","rb") as modelo_4:
    modelo_td4 = pickle.load(modelo_4)

prediccion = modelo_td4.predict(X)
df_final["Clasificación"]= prediccion
Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[6,7]]
Rubrica_df.columns=["Perspectiva", "Clasificación"]
Rubrica_df["Perspectiva"]=Rubrica_df["Perspectiva"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.08,
    2: 2.17,
    3: 3.25,
    4: 4.33,
    5: 5.42,
    6: 6.50
}
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,8]]
Puntaje_df.columns=["ID","Perspectiva"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"I{last_row-len(pred_s)}", values)


C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1733655343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Clasificación"]= prediccion
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\1733655343.py:35: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Puntaje.update(f"I{last_row-len(pred_s)}", values)


{'spreadsheetId': '1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y',
 'updatedRange': 'Sheet1!I277:J277',
 'updatedRows': 1,
 'updatedColumns': 2,
 'updatedCells': 2}

## Población objetivo

In [513]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn2=int(RN_df.loc[:,"DesgloseDeSujetosDeDerecho"])-3


C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\253484010.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
C:\Users\pablo\AppData\Local\Temp\ipykernel_43760\253484010.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  rn2=int(RN_df.loc[:,"DesgloseDeSujetosDeDerecho"])-3


In [514]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
Desglose=libro.get_worksheet(4)
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","HanRealizadoAlgúnDiagnósticoQueLePermitaConocerEnProfundidadLaSituaciónDeSuPoblaciónObjetivo"]]
df.columns=["ID","Estudio"]
df=df.iloc[rn:,:]
Desglose_d = Desglose.get_all_values()
Desglose_df=pd.DataFrame(Desglose_d)
Desglose_df.columns=Desglose_df.iloc[0,:]
Desglose_df=Desglose_df.iloc[2:,:]
Desglose_df=Desglose_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Edad","NoHombres","NoMujeres"]]
Desglose_df.columns=["ID","Edad","Hombres","Mujeres"]
Desglose_df=Desglose_df.iloc[rn2:,:]
df_final= pd.merge(df,Desglose_df, on="ID", how="inner")
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='PoblacionObj', how='inner')
# df_final= pd.merge(df_sfinal,df_2, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,6,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['Estudio'] = df_final['Estudio'].apply(quitar_acentos)
df_final['Estudio'] = df_final['Estudio'].astype(str)

df_final = df_final.dropna()
text = list(df_final['Estudio'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Estudio'] = corpus
df_final


APIError: APIError: [429]: Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:172469642267'.

In [ ]:
with open ("RF_TD5.pkl","rb") as modelo_5:
    modelo_td5 = pickle.load(modelo_5)

X = df_final.loc[:,['Estudio', 'Edad', 'Hombres', 'Mujeres']]

prediccion = modelo_td5.predict(X)
df_final["Clasificación"]= prediccion
Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[8,9]]
Rubrica_df.columns=["PoblacionObj","Clasificación"]
Rubrica_df["PoblacionObj"]=Rubrica_df["PoblacionObj"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.08,
    2: 2.17,
    3: 3.25,
    4: 4.33,
    5: 5.42,
    6: 6.5
}
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,10]]
Puntaje_df.columns=["ID","PoblacionObj"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"K{last_row-len(pred_s)}", values)



## Generación del conocimiento

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn2=int(RN_df.loc[:,"Actividades"])-3
print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
Actividades=libro.get_worksheet(5)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","RecolectanInformaciónDeLosTitularesDeResponsabilidadesfamiliasComunidadesOrganizacionesDeLaSociedadCivilOSCSindicatosYDeObligacionesinstitucionesPúblicasDelEstadoYLosGobernantes"]]
df.columns=["ID","Recolectan"]
df=df.iloc[rn:,:]
Actividades_d = Actividades.get_all_values()
Actividades_df=pd.DataFrame(Actividades_d)
Actividades_df.columns=Actividades_df.iloc[0,:]
Actividades_df=Actividades_df.iloc[2:,:]
Actividades_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","MediosDeVerificación"]]
Actividades_df["MediosDeVerificación"] = Actividades_df["MediosDeVerificación"].str.replace('\n', ' ', regex=True)
df2=Actividades_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","MediosDeVerificación"]]
df2.columns=["ID","MediosDeVerificación"]
df2=df2.iloc[rn2:,:]
df_final= pd.merge(df,df2, on="ID", how="inner")
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='GeneracionConocimiento', how='inner')
# df_final= pd.merge(df_sfinal,df_2, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['Recolectan'] = df_final['Recolectan'].apply(quitar_acentos)
df_final['Recolectan'] = df_final['Recolectan'].astype(str)
df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].apply(quitar_acentos)
df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].astype(str)
text = list(df_final['Recolectan'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Recolectan'] = corpus
text = list(df_final['MediosDeVerificación'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['MediosDeVerificación'] = corpus
df_final

In [ ]:
X_text1=df_final['Recolectan'] = df_final['Recolectan'].apply(quitar_acentos)
X_text2=df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].apply(quitar_acentos)

with open("tfidf1_model_td6.pkl", "rb") as f:  
    tfidf1_td6_loaded = pickle.load(f)  

with open("tfidf2_model_td6.pkl", "rb") as f:  
    tfidf2_td6_loaded = pickle.load(f)  

with open ("RFbuenoTD6.pkl","rb") as modelo_6:
    modelo_td6 = pickle.load(modelo_6)


X_text1_t=tfidf1_td6_loaded.transform(X_text1)
X_text2_t=tfidf2_td6_loaded.transform(X_text2)

X_combined = hstack([X_text1_t, X_text2_t])

prediccion = modelo_td6.predict(X_combined)
df_final["Clasificación"]= prediccion

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[10,11]]
Rubrica_df.columns=["GeneracionConocimiento", "Clasificación"]
Rubrica_df["GeneracionConocimiento"]=Rubrica_df["GeneracionConocimiento"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.08,
    2: 2.17,
    3: 3.25,
    4: 4.33,
    5: 5.42,
    6: 6.5
}
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,13]]
Puntaje_df.columns=["ID","GeneracionConocimiento"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"N{last_row-len(pred_s)}", values)



## Disponibilidad de información

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn2=int(RN_df.loc[:,"Actividades"])-3
print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
Actividades=libro.get_worksheet(5)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","CómoSeleccionanALosSujetosDeDerecho","QuéTipoDeInformaciónLesSolicitanALasPersonasAtendidasPorPrimeraVez","CómoLlevaACaboElSeguimientoDeLasDistintasAccionesYResultadosGeneradosEnElMarcoDeSusIntervenciones2","IndicadoresProposito","RecolectanInformaciónDeLosTitularesDeResponsabilidadesfamiliasComunidadesOrganizacionesDeLaSociedadCivilOSCSindicatosYDeObligacionesinstitucionesPúblicasDelEstadoYLosGobernantes","EnCasoDeQueSíquéTipoDeInformaciónLesSolicitanConQuéPeriodicidadCuentanConInstrumentosDeRecolecciónEstandarizadosParaRecolectarEstaInformación"]]
df.columns=["ID","Selección","InformaciónSolicitada","Seguimiento","IndicadoresProposito","RecollecionTitulares","InfoPeriodicidad"]
df=df.iloc[rn:,:]
Actividades_d = Actividades.get_all_values()
Actividades_df=pd.DataFrame(Actividades_d)
Actividades_df.columns=Actividades_df.iloc[0,:]
Actividades_df=Actividades_df.iloc[2:,:]
Actividades_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","MediosDeVerificación"]]
Actividades_df["MediosDeVerificación"] = Actividades_df["MediosDeVerificación"].str.replace('\n', ' ', regex=True)
df2=Actividades_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","MediosDeVerificación"]]
df2.columns=["ID","MediosDeVerificación"]
df2=df2.iloc[rn2:,:]
df_final= pd.merge(df,df2, on="ID", how="inner")
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='DisponibilidadInfo', how='inner')
# df_final= pd.merge(df_sfinal,df_2, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,6,7,8,9,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}
def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['Selección'] = df_final['Selección'].apply(quitar_acentos)
df_final['Selección'] = df_final['Selección'].astype(str)
df_final['InformaciónSolicitada'] = df_final['InformaciónSolicitada'].apply(quitar_acentos)
df_final['InformaciónSolicitada'] = df_final['InformaciónSolicitada'].astype(str)
df_final['Seguimiento'] = df_final['Seguimiento'].apply(quitar_acentos)
df_final['Seguimiento'] = df_final['Seguimiento'].astype(str)
df_final['IndicadoresProposito'] = df_final['IndicadoresProposito'].apply(quitar_acentos)
df_final['IndicadoresProposito'] = df_final['IndicadoresProposito'].astype(str)
df_final['RecollecionTitulares'] = df_final['RecollecionTitulares'].apply(quitar_acentos)
df_final['RecollecionTitulares'] = df_final['RecollecionTitulares'].astype(str)
df_final['InfoPeriodicidad'] = df_final['InfoPeriodicidad'].apply(quitar_acentos)
df_final['InfoPeriodicidad'] = df_final['InfoPeriodicidad'].astype(str)
df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].apply(quitar_acentos)
df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].astype(str)
text = list(df_final['Selección'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Selección'] = corpus
text = list(df_final['InformaciónSolicitada'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['InformaciónSolicitada'] = corpus
text = list(df_final['Seguimiento'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Seguimiento'] = corpus
text = list(df_final['IndicadoresProposito'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['IndicadoresProposito'] = corpus
text = list(df_final['RecollecionTitulares'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['RecollecionTitulares'] = corpus
text = list(df_final['InfoPeriodicidad'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['InfoPeriodicidad'] = corpus
text = list(df_final['MediosDeVerificación'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['MediosDeVerificación'] = corpus

df_final


In [ ]:
with open("tfidf1_model_td7.pkl", "rb") as f:  
    tfidf1_td7_loaded = pickle.load(f)  

with open("tfidf2_model_td7.pkl", "rb") as f:  
    tfidf2_td7_loaded = pickle.load(f)  

with open("tfidf3_model_td7.pkl", "rb") as f:  
    tfidf3_td7_loaded = pickle.load(f)  

with open("tfidf4_model_td7.pkl", "rb") as f:  
    tfidf4_td7_loaded = pickle.load(f)  

with open("tfidf5_model_td7.pkl", "rb") as f:  
    tfidf5_td7_loaded = pickle.load(f)  

with open("tfidf6_model_td7.pkl", "rb") as f:  
    tfidf6_td7_loaded = pickle.load(f)  

with open("tfidf7_model_td7.pkl", "rb") as f:  
    tfidf7_td7_loaded = pickle.load(f)  

with open ("RFbuenoTD7.pkl","rb") as modelo_7:
    modelo_td7 = pickle.load(modelo_7)



X_text1=df_final['Selección'] = df_final['Selección'].apply(quitar_acentos)
X_text2=df_final['InformaciónSolicitada'] = df_final['InformaciónSolicitada'].apply(quitar_acentos)
X_text3=df_final['Seguimiento'] = df_final['Seguimiento'].apply(quitar_acentos)
X_text4=df_final['IndicadoresProposito'] = df_final['IndicadoresProposito'].apply(quitar_acentos)
X_text5=df_final['RecollecionTitulares'] = df_final['RecollecionTitulares'].apply(quitar_acentos)
X_text6=df_final['InfoPeriodicidad'] = df_final['InfoPeriodicidad'].apply(quitar_acentos)
X_text7=df_final['MediosDeVerificación'] = df_final['MediosDeVerificación'].apply(quitar_acentos)

X_text1_t=tfidf1_td7_loaded.transform(X_text1)
X_text2_t=tfidf2_td7_loaded.transform(X_text2)
X_text3_t=tfidf3_td7_loaded.transform(X_text3)
X_text4_t=tfidf4_td7_loaded.transform(X_text4)
X_text5_t=tfidf5_td7_loaded.transform(X_text5)
X_text6_t=tfidf6_td7_loaded.transform(X_text6)
X_text7_t=tfidf7_td7_loaded.transform(X_text7)


X_combined = hstack([X_text1_t, X_text2_t,X_text3_t, X_text4_t,X_text5_t, X_text6_t,X_text7_t])

prediccion = modelo_td7.predict(X_combined)
df_final["Clasificación"]=prediccion
df_final

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[12,13]]
Rubrica_df.columns=["DisponibilidadInfo", "Clasificación"]
Rubrica_df["DisponibilidadInfo"]=Rubrica_df["DisponibilidadInfo"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s = pd.merge(df_final,Rubrica_df, on="Clasificación",how="inner")
mapeo = {
    0: 0.00,
    1: 1.08,
    2: 2.17,
    3: 3.25,
    4: 4.33,
    5: 5.42,
    6: 6.5
}

Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,15]]
Puntaje_df.columns=["ID","DisponibilidadInfo"]
pred_s["Puntaje"]=pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"P{last_row-len(pred_s)}", values)


## Uso del conocimiento para la toma de decisiones

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY")
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns = SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df = SolicitudDeInversiónSocialFocal_df.iloc[1:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","ConsideraRealizarUnaSiguienteEtapaDeSuProyecto"]]
df.columns=["ID", "Respuesta"]
df_final=df.iloc[rn:,:]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto

# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='ConocimientoDecisiones', how='inner')
# df_final= pd.merge(df_sfinal,df, on="ID", how="inner")
# df_final=df_final.iloc[:,[3,2]]
df_final['Respuesta'] = df_final['Respuesta'].apply(quitar_acentos)
df_final['Respuesta'] = df_final['Respuesta'].astype(str)
# df_final['Clasificación'] = df_final['Clasificación'].replace(1, 0)
text = list(df_final['Respuesta'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Respuesta'] = corpus
X = df_final['Respuesta']

In [ ]:
with open ("RFbuenoTD8.pkl","rb") as modelo_8:
    modelo_td8 = pickle.load(modelo_8)

prediccion = modelo_td8.predict(X)
df_final["Clasificación"]= prediccion


Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[14,15]]
Rubrica_df.columns=["ConocimientoDecisiones", "Clasificación"]
Rubrica_df["ConocimientoDecisiones"]=Rubrica_df["ConocimientoDecisiones"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.17,
    2: 2.33,
    3: 3.50,
    4: 4.67,
    5: 5.83,
    6: 7
}
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,17]]
Puntaje_df.columns=["ID","ConocimientoDecisiones"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"R{last_row-len(pred_s)}", values)



## Recursos para resultados

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","EnlistaLosGastosEnRecursosHumanos","DescripciónDeGastosEnRecursosHumanos","DescripciónDeGastosEnInsumosMateriales","DescripciónDeGastosEnServicios","EnlistaLosGastosParaInmueble","DescripciónDeGastosParaInmuebles","MontoDeGastosParaTrasladosYViáticos","EnlistaLosGastosEnTrasladosYViáticos","DescripciónDeGastosEnTrasladosYViáticos","DescripciónDeGastosEnMobiliarioYEquipo","TipoSolicitud"]]
df.columns=["ID","GastosRh","DescripcionGastosRh","DescripcionGastosInsumos","DescripcionGastosServicios","GastosInmueble","DescripcionGastosInmueble","MontoViaticos","GastosViaticos","DescripcionGastosViaticos","DescripcionGastosEquipo","TipoSolicitud"]
df_final=df.iloc[rn:,:]


# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='RecursosResultados', how='inner')
# df_final= pd.merge(df_sfinal,df, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,6,7,8,9,10,11,12,13,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['GastosRh'] = df_final['GastosRh'].apply(quitar_acentos)
df_final['GastosRh'] = df_final['GastosRh'].astype(str)
df_final['DescripcionGastosRh'] = df_final['DescripcionGastosRh'].apply(quitar_acentos)
df_final['DescripcionGastosRh'] = df_final['DescripcionGastosRh'].astype(str)
df_final['DescripcionGastosInsumos'] = df_final['DescripcionGastosInsumos'].apply(quitar_acentos)
df_final['DescripcionGastosInsumos'] = df_final['DescripcionGastosInsumos'].astype(str)
df_final['DescripcionGastosServicios'] = df_final['DescripcionGastosServicios'].apply(quitar_acentos)
df_final['DescripcionGastosServicios'] = df_final['DescripcionGastosServicios'].astype(str)
df_final['GastosInmueble'] = df_final['GastosInmueble'].apply(quitar_acentos)
df_final['GastosInmueble'] = df_final['GastosInmueble'].astype(str)
df_final['DescripcionGastosInmueble'] = df_final['DescripcionGastosInmueble'].apply(quitar_acentos)
df_final['DescripcionGastosInmueble'] = df_final['DescripcionGastosInmueble'].astype(str)
df_final['GastosViaticos'] = df_final['GastosViaticos'].apply(quitar_acentos)
df_final['GastosViaticos'] = df_final['GastosViaticos'].astype(str)
df_final['DescripcionGastosViaticos'] = df_final['DescripcionGastosViaticos'].apply(quitar_acentos)
df_final['DescripcionGastosViaticos'] = df_final['DescripcionGastosViaticos'].astype(str)
df_final['DescripcionGastosEquipo'] = df_final['DescripcionGastosEquipo'].apply(quitar_acentos)
df_final['DescripcionGastosEquipo'] = df_final['DescripcionGastosEquipo'].astype(str)
df_final['TipoSolicitud'] = df_final['TipoSolicitud'].apply(quitar_acentos)
df_final['TipoSolicitud'] = df_final['TipoSolicitud'].astype(str)

text = list(df_final['GastosRh'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['GastosRh'] = corpus
text = list(df_final['DescripcionGastosRh'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosRh'] = corpus
text = list(df_final['DescripcionGastosInsumos'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosInsumos'] = corpus
text = list(df_final['DescripcionGastosServicios'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosServicios'] = corpus
text = list(df_final['GastosInmueble'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['GastosInmueble'] = corpus
text = list(df_final['DescripcionGastosInmueble'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosInmueble'] = corpus
text = list(df_final['GastosViaticos'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['GastosViaticos'] = corpus
text = list(df_final['DescripcionGastosViaticos'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosViaticos'] = corpus
text = list(df_final['DescripcionGastosEquipo'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['DescripcionGastosEquipo'] = corpus
text = list(df_final['TipoSolicitud'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['TipoSolicitud'] = corpus
df_final['MontoViaticos'] = df_final['MontoViaticos'].astype(str)
df_final['MontoViaticos'] = df_final['MontoViaticos'].str.replace(r'\D', '', regex=True)
df_final['MontoViaticos'] = pd.to_numeric(df_final['MontoViaticos'], errors='coerce').fillna(0).astype(int)
df_final

In [ ]:
with open("tfidf1_model_td9.pkl", "rb") as f:  
    tfidf1_td9_loaded = pickle.load(f)  

with open("tfidf2_model_td9.pkl", "rb") as f:  
    tfidf2_td9_loaded = pickle.load(f)  

with open("tfidf3_model_td9.pkl", "rb") as f:  
    tfidf3_td9_loaded = pickle.load(f)  

with open("tfidf4_model_td9.pkl", "rb") as f:  
    tfidf4_td9_loaded = pickle.load(f)  

with open("tfidf5_model_td9.pkl", "rb") as f:  
    tfidf5_td9_loaded = pickle.load(f)  

with open("tfidf6_model_td9.pkl", "rb") as f:  
    tfidf6_td9_loaded = pickle.load(f)

with open("scaler_model_td9.pkl", "rb") as f:  
    scaler_td9_loaded = pickle.load(f)  

with open("tfidf8_model_td9.pkl", "rb") as f:  
    tfidf8_td9_loaded = pickle.load(f)  

with open("tfidf9_model_td9.pkl", "rb") as f:  
    tfidf9_td9_loaded = pickle.load(f)  

with open("tfidf10_model_td9.pkl", "rb") as f:  
    tfidf10_td9_loaded = pickle.load(f)  

with open ("RFbuenoTD9.pkl","rb") as modelo_9:
    modelo_td9 = pickle.load(modelo_9)

X_text1=df_final['GastosRh'] = df_final['GastosRh'].apply(quitar_acentos)
X_text2=df_final['DescripcionGastosRh'] = df_final['DescripcionGastosRh'].apply(quitar_acentos)
X_text3=df_final['DescripcionGastosInsumos'] = df_final['DescripcionGastosInsumos'].apply(quitar_acentos)
X_text4=df_final['DescripcionGastosServicios'] = df_final['DescripcionGastosServicios'].apply(quitar_acentos)
X_text5=df_final['GastosInmueble'] = df_final['GastosInmueble'].apply(quitar_acentos)
X_text6=df_final['DescripcionGastosInmueble'] = df_final['DescripcionGastosInmueble'].apply(quitar_acentos)
x_num = df_final[['MontoViaticos']]
X_text8=df_final['DescripcionGastosEquipo'] = df_final['DescripcionGastosEquipo'].apply(quitar_acentos)
X_text9=df_final['DescripcionGastosViaticos'] = df_final['DescripcionGastosViaticos'].apply(quitar_acentos)
X_text10=df_final['TipoSolicitud'] = df_final['TipoSolicitud'].apply(quitar_acentos)


X_text1_t=tfidf1_td9_loaded.transform(X_text1)
X_text2_t=tfidf2_td9_loaded.transform(X_text2)
X_text3_t=tfidf3_td9_loaded.transform(X_text3)
X_text4_t=tfidf4_td9_loaded.transform(X_text4)
X_text5_t=tfidf5_td9_loaded.transform(X_text5)
X_text6_t=tfidf6_td9_loaded.transform(X_text6)
x_num_t=scaler_td9_loaded.transform(x_num)
X_text8_t=tfidf8_td9_loaded.transform(X_text8)
X_text9_t=tfidf9_td9_loaded.transform(X_text9)
X_text10_t=tfidf10_td9_loaded.transform(X_text10)

X_combined = hstack([X_text1_t, X_text2_t, X_text3_t, X_text4_t, X_text5_t, X_text6_t, X_text8_t, X_text9_t, X_text10_t,x_num_t])

prediccion = modelo_td9.predict(X_combined)
df_final["Clasificación"]= prediccion
df_final

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[16,17]]
Rubrica_df.columns=["RecursosResultados", "Clasificación"]
Rubrica_df["RecursosResultados"]=Rubrica_df["RecursosResultados"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
pred_s
Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"U{last_row-len(pred_s)}", values)




## Planeación

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","FundamentaDeManeraBreveCómoEsQueElConjuntoDeProductosServiciosYCualquierOtroComponenteDeSusIntervencionesFavoreceráElCambioEsperado","SusActividadesImplicanElInvolucramientoDeLosTitularesDeResponsabilidadesfamiliasComunidadesOrganizacionesDeLaSociedadCivilOSCSindicatosYDeObligacionesinstitucionesPúblicasDelEstadoYLosGobernantesQuéSeRealizaConEllosasQuéBuscaGenerarEnLosMismos"]]
df.columns=["ID","IntervencionBeneficio","InvolucramientoTitulares"]
df_final=df.iloc[rn:,:]
df_final
# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Planeacion', how='inner')
# df_final= pd.merge(df_sfinal,df, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['IntervencionBeneficio'] = df_final['IntervencionBeneficio'].apply(quitar_acentos)
df_final['IntervencionBeneficio'] = df_final['IntervencionBeneficio'].astype(str)
df_final['InvolucramientoTitulares'] = df_final['InvolucramientoTitulares'].apply(quitar_acentos)
df_final['InvolucramientoTitulares'] = df_final['InvolucramientoTitulares'].astype(str)
text = list(df_final['IntervencionBeneficio'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['IntervencionBeneficio'] = corpus
text = list(df_final['InvolucramientoTitulares'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['InvolucramientoTitulares'] = corpus

X_text1=df_final['IntervencionBeneficio'] = df_final['IntervencionBeneficio'].apply(quitar_acentos)
X_text2=df_final['InvolucramientoTitulares'] = df_final['InvolucramientoTitulares'].apply(quitar_acentos)

In [ ]:
with open("tfidf1_model_td10.pkl", "rb") as f:  
    tfidf1_td10_loaded = pickle.load(f)  

with open("tfidf2_model_td10.pkl", "rb") as f:  
    tfidf2_td10_loaded = pickle.load(f)  

with open ("MNB_buenoTD10.pkl","rb") as modelo_10:
    modelo_td10 = pickle.load(modelo_10)

X_text1_t=tfidf1_td10_loaded.transform(X_text1)
X_text2_t=tfidf2_td10_loaded.transform(X_text2)

X_combined = hstack([X_text1_t, X_text2_t])

prediccion = modelo_td10.predict(X_combined)
df_final["Clasificación"]= prediccion


Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[18,19]]
Rubrica_df.columns=["Planeacion", "Clasificación"]
Rubrica_df["Planeacion"]=Rubrica_df["Planeacion"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}


Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,22]]
Puntaje_df.columns=["ID","Planeacion"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]

last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"W{last_row-len(pred_s)}", values)


## Alianzas de complementariedad

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original


RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"CoinversoresTabla"])-3
rn2=int(RN_df.loc[:,"AliadosDelProyecto"])-3

print(rn)

In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
CoinversoresTabla=libro.get_worksheet(11)
AliadosDelProyecto=libro.get_worksheet(2)
CoinversoresTabla_d = CoinversoresTabla.get_all_values()
CoinversoresTabla_df=pd.DataFrame(CoinversoresTabla_d)
CoinversoresTabla_df.columns=CoinversoresTabla_df.iloc[0,:]
CoinversoresTabla_df=CoinversoresTabla_df.iloc[2:,:]
df=CoinversoresTabla_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Coinversor","MontoCoinversor"]]
df.columns=["ID","Coinversor","MontoCoinversor"]
df=df.iloc[rn:,:]
AliadosDelProyecto=libro.get_worksheet(2)
AliadosDelProyecto_d = AliadosDelProyecto.get_all_values()
AliadosDelProyecto_df=pd.DataFrame(AliadosDelProyecto_d)
AliadosDelProyecto_df.columns=AliadosDelProyecto_df.iloc[0,:]
AliadosDelProyecto_df=AliadosDelProyecto_df.iloc[2:,:]
df2=AliadosDelProyecto_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","NombreAliado","SectorAliado","QuéPapelTieneElAliadoEnElProyecto"]]
df2.columns=["ID","NombreAliado","SectorAliado","QuéPapelTieneElAliadoEnElProyecto"]
df2=df2.iloc[rn2:,:]
df_final=pd.merge(df,df2, on="ID", how="inner")

# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Alianzas', how='inner')
# df_final= pd.merge(df_sfinal,df3, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,6,7,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['Coinversor'] = df_final['Coinversor'].apply(quitar_acentos)
df_final['Coinversor'] = df_final['Coinversor'].astype(str)
df_final['NombreAliado'] = df_final['NombreAliado'].apply(quitar_acentos)
df_final['NombreAliado'] = df_final['NombreAliado'].astype(str)
df_final['SectorAliado'] = df_final['SectorAliado'].apply(quitar_acentos)
df_final['SectorAliado'] = df_final['SectorAliado'].astype(str)
df_final['QuéPapelTieneElAliadoEnElProyecto'] = df_final['QuéPapelTieneElAliadoEnElProyecto'].apply(quitar_acentos)
df_final['QuéPapelTieneElAliadoEnElProyecto'] = df_final['QuéPapelTieneElAliadoEnElProyecto'].astype(str)
text = list(df_final['Coinversor'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['Coinversor'] = corpus
text = list(df_final['NombreAliado'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['NombreAliado'] = corpus
text = list(df_final['SectorAliado'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['SectorAliado'] = corpus
text = list(df_final['QuéPapelTieneElAliadoEnElProyecto'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['QuéPapelTieneElAliadoEnElProyecto'] = corpus
df_final['MontoCoinversor'] = df_final['MontoCoinversor'].astype(str) 
df_final['MontoCoinversor'] = df_final['MontoCoinversor'].str.replace(r'\D', '', regex=True)
df_final['MontoCoinversor'] = pd.to_numeric(df_final['MontoCoinversor'], errors='coerce').fillna(0).astype(int)


In [ ]:
with open("tfidf1_model_td11.pkl", "rb") as f:  
    tfidf1_td11_loaded = pickle.load(f)  

with open("tfidf2_model_td11.pkl", "rb") as f:  
    tfidf2_td11_loaded = pickle.load(f)  

with open("tfidf3_model_td11.pkl", "rb") as f:  
    tfidf3_td11_loaded = pickle.load(f)  

with open("tfidf4_model_td11.pkl", "rb") as f:  
    tfidf4_td11_loaded = pickle.load(f) 

with open("scaler_model_td11.pkl", "rb") as f:  
    scaler_td11_loaded = pickle.load(f)  

X_text1 = df_final['Coinversor'].apply(quitar_acentos)
X_text2 = df_final['NombreAliado'].apply(quitar_acentos)
X_text3 = df_final['SectorAliado'].apply(quitar_acentos)
X_text4 = df_final['QuéPapelTieneElAliadoEnElProyecto'].apply(quitar_acentos)

df_final['MontoCoinversor'] = df_final['MontoCoinversor'].astype(str).str.replace(r'\D', '', regex=True)
df_final['MontoCoinversor'] = pd.to_numeric(df_final['MontoCoinversor'], errors='coerce').fillna(0).astype(int)
X_num = df_final[['MontoCoinversor']] 

X_text1_t=tfidf1_td11_loaded.transform(X_text1)
X_text2_t=tfidf2_td11_loaded.transform(X_text2)
X_text3_t=tfidf3_td11_loaded.transform(X_text3)
X_text4_t=tfidf4_td11_loaded.transform(X_text4)
x_num_t=scaler_td11_loaded.transform(X_num)

with open ("RF_buenoTD11.pkl","rb") as modelo_11:
    modelo_td11 = pickle.load(modelo_11)


X_combined=hstack([X_text1_t, X_text2_t, X_text3_t, X_text4_t, x_num_t])

prediccion = modelo_td11.predict(X_combined)
df_final["Clasificación"]= prediccion

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[20,21]]
Rubrica_df.columns=["Alianzas", "Clasificación"]
Rubrica_df["Alianzas"]=Rubrica_df["Alianzas"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}


Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,25]]
Puntaje_df.columns=["ID","Alianzas"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"Z{last_row-len(pred_s)}", values)

## Participación

In [ ]:
sheet_id="1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY"

libro = client.open_by_key(sheet_id) #Original

RN=libro.get_worksheet(12)
RN_d = RN.get_all_values()
RN_df=pd.DataFrame(RN_d)
RN_df.columns=RN_df.iloc[0,:]
RN_df=RN_df.iloc[1:,:]
rn=int(RN_df.loc[:,"SolicitudDeInversiónSocialFocal"])-2
rn2=int(RN_df.loc[:,"DesgloseDeSujetosDeDerecho"])-3


In [ ]:
libro = client.open_by_key("1r6mOjvi5ITuIgVJn9ec6hNph_ECWywA90uJxZnNYYCY") #Original
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal=libro.get_worksheet(1)
SolicitudDeInversiónSocialFocal_d = SolicitudDeInversiónSocialFocal.get_all_values()
SolicitudDeInversiónSocialFocal_df=pd.DataFrame(SolicitudDeInversiónSocialFocal_d)
SolicitudDeInversiónSocialFocal_df.columns=SolicitudDeInversiónSocialFocal_df.iloc[0,:]
SolicitudDeInversiónSocialFocal_df=SolicitudDeInversiónSocialFocal_df.iloc[1:,:]
df=SolicitudDeInversiónSocialFocal_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","DeQuéManeraSeInvolucranLosSujetosDeDerechoEnElDesarrolloDeLasActividades","SusActividadesImplicanElInvolucramientoDeLosTitularesDeResponsabilidadesfamiliasComunidadesOrganizacionesDeLaSociedadCivilOSCSindicatosYDeObligacionesinstitucionesPúblicasDelEstadoYLosGobernantesQuéSeRealizaConEllosasQuéBuscaGenerarEnLosMismos"]]
df.columns=["ID","InvolucranSujetosDerecho","InvolucramientoTitulares"]
df=df.iloc[rn:,:]

Desglose=libro.get_worksheet(4)
Desglose_d = Desglose.get_all_values()
Desglose_df=pd.DataFrame(Desglose_d)
Desglose_df.columns=Desglose_df.iloc[0,:]
Desglose_df=Desglose_df.iloc[2:,:]
Desglose_df=Desglose_df.loc[:,["SolicitudDeInversiónSocialFocal_Id","Edad","NoHombres","NoMujeres"]]
Desglose_df.columns=["ID","Edad","Hombres","Mujeres"]
Desglose_df=Desglose_df.iloc[rn2:,:]
Desglose_df

df_final=pd.merge(df,Desglose_df, on="ID", how="inner")

# df_sfinal = pd.merge(Puntaje_df,Rubrica_df, on='Participacion', how='inner')
# df_final= pd.merge(df_sfinal,df3, on="ID", how="inner")
# df_final = df_final.iloc[:,[0,3,4,5,6,7,2]]
acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
           'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', "Ñ":"N", "ñ":"n"}

def quitar_acentos(texto):
    for acentuada, sin_acento in acentos.items():
        texto = texto.replace(acentuada, sin_acento)
    return texto
df_final['InvolucranSujetosDerecho'] = df_final['InvolucranSujetosDerecho'].apply(quitar_acentos)
df_final['InvolucranSujetosDerecho'] = df_final['InvolucranSujetosDerecho'].astype(str)
df_final['InvolucramientoTitulares'] = df_final['InvolucramientoTitulares'].apply(quitar_acentos)
df_final['InvolucramientoTitulares'] = df_final['InvolucramientoTitulares'].astype(str)
text = list(df_final['InvolucranSujetosDerecho'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['InvolucranSujetosDerecho'] = corpus
text = list(df_final['InvolucramientoTitulares'])
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(text)):

    r = re.sub('[^a-zA-Z]', ' ', text[i])

    r = r.lower()

    r = r.split()

    r = [word for word in r if word not in stopwords.words('spanish')]

    r = [lemmatizer.lemmatize(word) for word in r]

    r = ' '.join(r)

    corpus.append(r)
df_final['InvolucramientoTitulares'] = corpus


In [ ]:
with open("tfidf_model_td12.pkl", "rb") as f:  
    tfidf_td12_loaded = pickle.load(f)  

with open("scaler_model_td12.pkl", "rb") as f:  
    scaler_td12_loaded = pickle.load(f)

with open ("RF_buenoTD12.pkl","rb") as modelo_12:
    modelo_td12 = pickle.load(modelo_12)


text_features = ['InvolucranSujetosDerecho', "InvolucramientoTitulares"]
numeric_features = ['Edad', 'Hombres', 'Mujeres']

X_text_t=tfidf_td12_loaded.transform(df_final[text_features].apply(lambda x: ' '.join(x.astype(str)), axis=1))
X_numeric_t=scaler_td12_loaded.transform(df_final[numeric_features])

X_combined = hstack([X_text_t, X_numeric_t])

prediccion = modelo_td12.predict(X_combined)
df_final["Clasificación"]= prediccion

Cálculo_Puntaje=client.open_by_key("1mqxPzWLSjj13hhM4VVJRTWGMsNI1R4-tHgFAbO81MJg") #Original
Rubrica=Cálculo_Puntaje.get_worksheet(1)
Rubrica_d = Rubrica.get_all_values()
Rubrica_df=pd.DataFrame(Rubrica_d)
Rubrica_df=Rubrica_df.iloc[1:,[22,23]]
Rubrica_df.columns=["Participacion", "Clasificación"]
Rubrica_df["Participacion"]=Rubrica_df["Participacion"].astype(str)
Rubrica_df['Clasificación'] = range(0, len(Rubrica_df))
pred_s=pd.merge(df_final,Rubrica_df, on='Clasificación', how='inner')
mapeo = {
    0: 0.00,
    1: 1.67,
    2: 3.33,
    3: 5.00,
    4: 6.67,
    5: 8.33,
    6: 10
}

Puntajes=client.open_by_key("1h_RMQImx4eMdcOPX_4oS9Umac168xA45hxXbo3uaX2Y")
Puntaje=Puntajes.get_worksheet(0)
Puntaje_d = Puntaje.get_all_values()
Puntaje_df=pd.DataFrame(Puntaje_d)
Puntaje_df=Puntaje_df.iloc[2:,[0,27]]
Puntaje_df.columns=["ID","Participacion"]
pred_s["Puntaje"] = pred_s["Clasificación"].map(mapeo)
pred_s=pred_s.iloc[:,[-2,-1]]
last_row = len(Puntaje.get_all_values()) + 1 # Encuentra la primera fila vacía

values = pred_s.values.tolist()
Puntaje.update(f"AB{last_row-len(pred_s)}", values)

